# Lecture 3.1 : Matrix Product States (constructing, expecting & inspecting)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PhilipVinc/ComputationalQuantumPhysics/blob/main/Notebooks/3-TensorNetworks/3-aaa.ipynb)

Today we will be investigating the basics of representing a wavefunction as a Matrix Product State (MPS). In particular you will have to:
1. Convert a many-body wavefunction stored as a dense vector into an **MPS** using successive SVDs.
2. Read off **Schmidt values** and compute the **bipartite entanglement entropy** across all cuts.
3. Compute **expectation values** from the MPS and compare to ED.
4. Explore **compression** by truncating the bond dimension $\chi$ and quantify the encoding error .

Compared to the last 2 weeks, this time the lab exercise is somewhat more challenging, and there will be little physics but some more complex code. The algorithms you will have to implement where discussed this morning, and you will now have to convert them from _algebric formulas_ to code.  
My suggestions for today:
1. Think in small pieces. Do not write a huge function at once, instead try to split up your algorithms into smaller sub-blocks.
2. Always double check the shape of tensors by `print(tensor.shape)` to make sure that they indeed have the shape you expect.
3. When you're uncertain, think about how you can validate the result of your algorithm. For example, if you know your function `your_complex_function(...)` computes a matrix `A` in the middle which should be an isometry, you can check if `A` is indeed an isomtry. If it is, the problem is probably later. If it's not, the problem is before.
4. Remember that the simplest way to write tensor contractions is `np.einsum`. It's not the fastest, but it's easy. Start simple, then if you feel brave you can look for a more efficient implementation.

---

We will be working with the eigenstates of the **1D TFIM** with **open boundary conditions** (in the past we always had periodic boundary conditions!):
$$
H = -J \sum_{i=1}^{L-1} \sigma_i^z\sigma_{i+1}^z - h \sum_{i=1}^{L} \sigma_i^x .
$$

---

## How to use this notebook

You will see:
- **Assignment cells**: what you must implement.
- **Scaffold cells**: function skeletons with `TODO`.
- **Tests**: small checks you can run to validate your implementations. Passing tests is your “green light”. Be warned that a failing test means your function does not work, but a passing test does not mean it's 100% guaranteed to be correct! 
- **SOLUTION** cells: fully worked solutions (kept in the notebook so you can self-check).

Please try to implement first, then use the SOLUTION cells only if you get stuck.


## What is a “test”?

A **test** is a small piece of code that checks whether a function does what it should.
It helps you:
- catch bugs early,
- validate behavior without “eyeballing” plots.

A typical test uses `assert`:
- if the condition is **true**, nothing happens,
- if it is **false**, Python raises an `AssertionError`.

The idea of a test is to validate your function against some known value, or a value computed through a different code path.

### Example (warm-up test)

Below we define a silly function and then test it. It will fail. Try to fix it.


In [43]:
def add(a, b):
    # pretend we wrote something clever
    return a - b

# Tests (should pass silently)
print("Testing the add function...")
assert add(1, 2) == 3
assert add(-1, 1) == 0
print("Warm-up tests passed ✅")


Testing the add function...


AssertionError: 

## Setup: basic linear algebra and TFIM helpers

Here i provide some code to speed you up, to build the TFIM Hamiltonian

We represent a wavefunction $|\psi\rangle$ as a complex vector of length $2^L$,
in the computational basis ordered as $|s_1 s_2 \dots s_L\rangle$ with $s_i \in \{0,1\}$.

We will use *open boundary conditions*.

Run the next cell to define:
- Pauli matrices
- TFIM Hamiltonian builder
- ED ground state solver


In [44]:
import numpy as np
from scipy.sparse import kron, eye, csr_matrix
from scipy.sparse.linalg import eigsh

# Pauli matrices (as sparse)
sx = csr_matrix(np.array([[0, 1], [1, 0]], dtype=complex))
sy = csr_matrix(np.array([[0, -1j], [1j, 0]], dtype=complex))
sz = csr_matrix(np.array([[1, 0], [0, -1]], dtype=complex))
id2 = csr_matrix(np.eye(2, dtype=complex))

def kron_all(ops):
    """Kronecker product of a list of 2x2 operators (sparse version)."""
    out = csr_matrix(np.array([[1]], dtype=complex))
    for op in ops:
        out = kron(out, op, format='csr')
    return out

def op_on_site(op, i, L):
    """Place single-site operator `op` on site i (0-based) in a chain of length L."""
    ops = [id2]*L
    ops[i] = op
    return kron_all(ops)

def op_on_bond(op_i, i, op_j, j, L):
    """Place op_i on site i and op_j on site j (0-based)."""
    ops = [id2]*L
    ops[i] = op_i
    ops[j] = op_j
    return kron_all(ops)

def tfim_hamiltonian(L, J=1.0, h=1.0):
    """Sparse TFIM Hamiltonian with OBC: -J sum sz_i sz_{i+1} - h sum sx_i."""
    dim = 2**L
    H = csr_matrix((dim, dim), dtype=complex)

    for i in range(L-1):
        H = H + (-J * op_on_bond(sz, i, sz, i+1, L))
    for i in range(L):
        H = H + (-h * op_on_site(sx, i, L))
    return H

def ground_state_ed(H):
    """Return (E0, psi0) for a sparse Hamiltonian using Lanczos algorithm."""
    # Use sparse eigensolver for the lowest eigenvalue
    # k=1 means we want 1 eigenvalue, which='SA' means smallest algebraic
    evals, evecs = eigsh(H, k=1, which='SA')
    E0 = evals[0].real
    psi0 = evecs[:, 0]
    # normalize (just in case)
    psi0 = psi0 / np.linalg.norm(psi0)
    return E0, psi0

# 1. Convert a state vector to an MPS (successive SVD)

We will represent an MPS as a list of tensors:
- `MPS = list[np.ndarray]`
- `A[i]` has shape `(chi_left, d, chi_right)` with `d=2` for spin-1/2.
- Open boundaries: `chi_left=1` for the first tensor, `chi_right=1` for the last tensor.

You must implement a function `state_to_mps` with the following signature
```python
def state_to_mps(psi: np.ndarray, L: int, d: int=2, chi_max:int|None=None) ->
   tuple[list[np.ndarray],list[np.ndarray]]:
```
this function must therefore take as an input a vector `psi` of length $d^L$, as well as the system size $L$ and the local hilbert space dimension $d$. It also accepts an optional argument `chi_max` which signals the maximum bond dimension between two tensors. If None, there is no maximum.

The function will be used as 
```python
mps, schmidt = state_to_mps(psi, L, d=2, chi_max=None)
```
- The function should return two values:
  - `mps`: the MPS stored as a list of tensors `A[i]` as discussed above
  - `schmidt`: list of 1D arrays of singular values at each cut (length `L-1`)

### Algorithm sketch 
We discussed the algorithm in class. Feel free to look at the lecture notes to revise it. 
**I suggest you disable autocompletion in your editor, to avoid having the AI doing the work for you. Try to implement it yourself first.**
Start with `psi` reshaped into a matrix separating site 0 from the rest:
- reshape to `(d, d**(L-1))`, do SVD to get the `U S V` matrices. NOTE: Use `np.linalg.svd(..., full_matrices=False)`
- keep up to `chi_max` singular values. If `chi_max` is None keep all of them.
- identify from the lements of the svd the first MPS tensor `A[0]` and store it as `(1, d, chi)`. That's your first MPS tensor.
- store the singular values S
- combine `psi = S V†` into a single object that will be the 'wavefunction' for the rest of the system. This should have shape `(chi, d, d**(L-1))`.
- Recurisvely decompose this smaller psi.

**Tip:** at first, do not worry about truncating to chi_max and always keep the full matrix. Then, once you have something working, add this functionality

### Self-validation test you must pass
- There is, below, a test that checks that a very simple |10000> state is converted correctly. You can try with other states as well. 


In [5]:
def state_to_mps(psi, L, d=2, chi_max=None):
    """Convert a state vector psi (length d**L) into an open-boundary MPS via successive SVD.

    Returns
    -------
    mps : list[np.ndarray]
        A[i] has shape (chi_left, d, chi_right)
    schmidt : list[np.ndarray]
        singular values at each cut (length L-1)
    """
    # TODO: implement
    raise NotImplementedError


### Test 1: Product state validation

Before looking at the solution, test your implementation on a simple case where you know the answer!

A **product state** has no entanglement, so:
- Schmidt values at each cut should be `[1.0]` (only one non-zero singular value)
- Bond dimensions should all be `chi=1`
- MPS tensors should have shape `(1, 2, 1)` for all sites

In [8]:
# Test with a product state: all spins up |00...0>
L_test = 6
psi_product = np.zeros(2**L_test, dtype=complex)
psi_product[0] = 1.0  # |000000> state

# Convert to MPS
mps_prod, schmidt_prod = state_to_mps(psi_product, L=L_test, d=2, chi_max=None)

# Validation checks
print("Testing product state |00...0>:")
print(f"  Number of tensors: {len(mps_prod)} (expected {L_test})")

# Check tensor shapes - should all be (1, 2, 1) for a product state
for i, A in enumerate(mps_prod):
    chi_L, d, chi_R = A.shape
    print(f"  Tensor {i}: shape {A.shape}", end="")
    if i == 0:
        assert chi_L == 1, f"First tensor should have chi_left=1, got {chi_L}"
    if i == L_test - 1:
        assert chi_R == 1, f"Last tensor should have chi_right=1, got {chi_R}"
    # For product state, all bond dims should be 1
    assert chi_L == 1 and chi_R == 1, f"Product state should have chi=1, got ({chi_L}, {chi_R})"
    print(" ✓")

# Check Schmidt values - should all be [1.0] (single singular value)
print(f"  Number of Schmidt cuts: {len(schmidt_prod)} (expected {L_test-1})")
for i, s_vals in enumerate(schmidt_prod):
    print(f"  Cut {i}: {len(s_vals)} singular values, max={s_vals[0]:.6f}", end="")
    assert len(s_vals) == 1, f"Product state should have 1 singular value per cut, got {len(s_vals)}"
    assert abs(s_vals[0] - 1.0) < 1e-10, f"Singular value should be 1.0, got {s_vals[0]}"
    print(" ✓")

print("\nProduct state test passed ✅")

Testing product state |00...0>:
  Number of tensors: 6 (expected 6)
  Tensor 0: shape (1, 2, 1) ✓
  Tensor 1: shape (1, 2, 1) ✓
  Tensor 2: shape (1, 2, 1) ✓
  Tensor 3: shape (1, 2, 1) ✓
  Tensor 4: shape (1, 2, 1) ✓
  Tensor 5: shape (1, 2, 1) ✓
  Number of Schmidt cuts: 5 (expected 5)
  Cut 0: 1 singular values, max=1.000000 ✓
  Cut 1: 1 singular values, max=1.000000 ✓
  Cut 2: 1 singular values, max=1.000000 ✓
  Cut 3: 1 singular values, max=1.000000 ✓
  Cut 4: 1 singular values, max=1.000000 ✓

Product state test passed ✅


In [7]:
# =========================
# SOLUTION (B1): state_to_mps
# =========================
def state_to_mps(psi, L, d=2, chi_max=None):
    psi = np.asarray(psi, dtype=complex)
    assert psi.ndim == 1
    assert psi.shape[0] == d**L
    # normalize defensively
    nrm = np.linalg.norm(psi)
    if nrm == 0:
        raise ValueError("psi has zero norm")
    psi = psi / nrm

    mps = []
    schmidt = []

    # "rest" carries the effective wavefunction for remaining sites,
    # with a left bond dimension that grows as we go.
    rest = psi.reshape((d, d**(L-1)))  # left bond is implicitly 1

    chi_left = 1
    for site in range(L-1):
        # rest currently has shape (chi_left*d, d**(L-site-1))
        # For site=0, chi_left=1 so shape (d, d**(L-1)) as above.
        U, S, Vh = np.linalg.svd(rest, full_matrices=False)

        # Filter out near-zero singular values (numerical noise)
        # This is important for product states and low-entanglement states
        tol = 1e-14 * S[0] if len(S) > 0 else 1e-14
        chi_svd = np.sum(S > tol)  # number of non-negligible singular values
        
        # Apply truncation (chi_max) on top of noise filtering
        if chi_max is None:
            chi = chi_svd
        else:
            chi = min(int(chi_max), chi_svd)
        
        # Ensure at least one singular value is kept
        chi = max(chi, 1)

        U = U[:, :chi]
        S = S[:chi]
        Vh = Vh[:chi, :]

        schmidt.append(S.copy())

        # reshape U into (chi_left, d, chi_right)
        A = U.reshape((chi_left, d, chi))
        mps.append(A)

        # update rest = S * Vh  (absorb singular values to the right)
        rest = (S[:, None] * Vh)

        # prepare for next iteration:
        chi_left = chi
        # reshape to group next physical index with left bond
        rest = rest.reshape((chi_left * d, d**(L-site-2)))

    # last tensor: rest has shape (chi_left*d, 1) effectively,
    # but after loop ends, rest is shape (chi_left*d, d**0) = (chi_left*d, 1)
    A_last = rest.reshape((chi_left, d, 1))
    mps.append(A_last)

    return mps, schmidt

# 2. Convert an MPS to a dense vector

You now how have to implement a function `mps_to_state` to revert the `mps` to a dense vector representation. This will allow us to test that the conversions are correct, as we will be able to check that `state == mps_to_state(state_to_mps(state))`.

Implement a function that contracts an open-boundary MPS back into a full state vector of length `d**L`.
This is not efficient for big systems, but for `L<=12` it's fine and extremely useful for debugging and tests.

**Tip:** Implementing this is relatively straightforward. Consider all MPS tensors to be 3-tensors, even the first and the last, and start contracting left-to-right with a vector of shape (1,) initialised to 1.0. At the end of the chains of contractions, you will get a 'matrix' of shape (1,1), which you should reshape into a scalar of shape (,) and it will be your result.

**Test to pass:** Without truncation, the reconstruction error must be tiny. So you should verify that if you go back the overlap between your original state and the one obtained by converting twice is 1. 


To do:
- Once your code is verified and works, do the following: consider the ground-state of the TFIM model computed with ED for $h=1$ at a fixed system size $N$. 
  - Convert those states with MPS, and impose a maximum bond dimension $\chi$, and then back-convert them to a dense vector.
  - The state now MIGHT be different. Compute the overlap which will be a number in $[0,1]$ and plot it as a function of the bond dimension cutoff $\chi$. What do you learn?

In [9]:
def mps_to_state(mps):
    """Contract an open-boundary MPS into a full state vector.

    Parameters
    ----------
    mps : list[np.ndarray]
        A[i] has shape (chi_left, d, chi_right)

    Returns
    -------
    psi : np.ndarray
        complex vector of length d**L
    """
    # TODO: implement
    raise NotImplementedError


In [ ]:
# =========================
# SOLUTION (C1): mps_to_state
# =========================
def mps_to_state(mps):
    """Contract an open-boundary MPS into a full state vector.

    Strategy: contract tensors sequentially, keeping physical indices separate.
    After each contraction, we have shape (..., d_i, d_{i+1}, ..., chi_right).
    """
    if len(mps) == 0:
        return np.array([1.0+0j])

    # Start with first tensor: (1, d, chi) -> squeeze left bond -> (d, chi)
    result = np.asarray(mps[0], dtype=complex)
    result = result[0, :, :]  # Remove left bond dimension of 1, shape is now (d, chi)

    # Contract with remaining tensors
    for A in mps[1:]:
        A = np.asarray(A, dtype=complex)  # (chi_left, d, chi_right)
        # result has shape (...physical indices..., chi_left)
        # Contract on the bond: last axis of result with first axis of A
        result = np.tensordot(result, A, axes=([-1], [0]))
        # result now has shape (...physical indices..., d, chi_right)

    # result now has shape (d, d, d, ..., d, 1)
    # Remove the trailing bond dimension and flatten
    result = result[..., 0]  # Remove last dimension (chi_right = 1)
    psi = result.ravel()  # Flatten all physical indices into (d^L,)

    return psi

### Test: state → MPS → state must match state (if no truncation)

In [45]:
H = tfim_hamiltonian(10, J=1.0, h=1.0)
E0_ed, psi0_ed = ground_state_ed(H)

# Build MPS from ED state (no truncation)
mps_full, sch_full = state_to_mps(psi0_ed, L=L, d=2, chi_max=None)
psi_recon = mps_to_state(mps_full)

err = np.linalg.norm(psi0_ed - psi_recon)
# Note: global phase can differ; handle by phase alignment:
phase = np.vdot(psi_recon, psi0_ed)
if abs(phase) > 0:
    psi_recon_aligned = psi_recon * (phase/abs(phase))
    err = np.linalg.norm(psi0_ed - psi_recon_aligned)

print("Reconstruction error:", err)
assert err < 1e-10
print("Test passed ✅ (reconstruction is accurate)")


Reconstruction error: 6.887895173024952e-15
Test passed ✅ (reconstruction is accurate)


# 3. Computing entanglement entropy from the Schmidt values

From the SVD at cut $\ell$ you obtained singular values $s_k$.  
These are the **Schmidt coefficients**, so the reduced density matrix eigenvalues are:
$$
\lambda_k = \frac{s_k^2}{\sum_j s_j^2}.
$$
The von Neumann entropy is:
$$
S = -\sum_k \lambda_k \log \lambda_k .
$$

- You should implement `entanglement_entropy_from_singular_values(S)`

Write a function that takes a 1D array of arrays encoding the singular values and returns the entanglement entropy for every bipartition.
Be careful with zeros (ignore terms with $\lambda_k=0$).

- To make sure you get the correct result, pick a reasonable system size (N=14 or 16) and plot the entanglement entropy across the bipartitions for different values of the transverse field $h$ and compare with what you did last week.


In [ ]:
def entanglement_entropy_from_singular_values(svals):
    """Von Neumann entropy from singular values at one cut."""
    # TODO: implement
    raise NotImplementedError


In [13]:
# =========================
# SOLUTION (D1): entropy from Schmidt values
# =========================
def entanglement_entropy_from_singular_values(svals):
    s = np.asarray(svals, dtype=float)
    if s.size == 0:
        return 0.0
    w = s*s
    Z = w.sum()
    if Z == 0:
        return 0.0
    lam = w / Z
    lam = lam[lam > 0]
    return float(-(lam * np.log(lam)).sum())


In [14]:
# Compute entanglement profile S(ell) for ED-derived MPS
S_profile = np.array([entanglement_entropy_from_singular_values(s) for s in sch_full])

print("Cuts:", list(range(1, L)))
print("S(ell):", np.round(S_profile, 6))


Cuts: [1, 2, 3, 4, 5, 6, 7, 8, 9]
S(ell): [0.26486  0.328772 0.359307 0.374413 0.379052 0.374413 0.359307 0.328772
 0.26486 ]


# 4. Expectation values from MPS

You will compute $\langle \sigma_i^z \rangle$ from the MPS **without** reconstructing the full state.

You must implement the function `expect_one_site(mps, op, i)`

Given:
- `mps`: list of tensors `A[k]` with shape `(chiL, d, chiR)`
- `op`: a `d x d` operator
- `i`: site index (0-based)

Return $\langle \psi | O_i | \psi \rangle$.

**Hint** Look at what we discussed in class/the lecture notes, and use the environment contraction algorithm by assuming that the intermediate operators $W^i$ are just identities so we leave them out.
In particular, you can:
- Define a left environment `Lenv` starting from scalar `[[1]]`:
- For each site `k`, update:
  - if `k != i`: contract with identity on the physical leg
  - if `k == i`: contract with `op`
Similarly you could do right environments, but a single left-to-right contraction works for open MPS.

To check that your code is correct, you should compare against the ED baseline, where you convert your mps to a dense vector, you build the operator's sparse matrix representation, and you compute the result with ED we did in the past weeks.
That 'validation' function will be `expect_one_site_via_state(mps, op, i)` and will only work for small systems, while your `expect_one_site` should work for even very large system sizes.


In [ ]:
def expect_one_site(mps, op, i):
    """Compute <psi| op_i |psi> from an open-boundary MPS."""
    # TODO: implement (no reconstruction)
    raise NotImplementedError

def expect_one_site_via_state(mps, op, i):
    """Slow reference: reconstruct state and compute expectation (for testing)."""
    psi = mps_to_state(mps)
    O = ...
    # TODO: implement a simple algorithm
    raise NotImplementedError
    return np.vdot(psi, O @ psi).real

In [ ]:
# =========================
# SOLUTION (E1): expect_one_site via environments
# =========================
def expect_one_site(mps, op, i):
    L = len(mps)
    d = mps[0].shape[1]
    # Convert op to dense array for einsum (sparse arrays don't work well with einsum)
    if hasattr(op, 'toarray'):
        op = op.toarray()
    op = np.asarray(op, dtype=complex)
    assert op.shape == (d, d)
    assert 0 <= i < L

    # left environment: matrix on the current right-bond space
    # start with scalar 1 on bond dimension 1
    env = np.array([[1.0+0j]])  # shape (chiL, chiL) for current site

    for k, A in enumerate(mps):
        A = np.asarray(A, dtype=complex)  # (chiL, d, chiR)
        chiL, dA, chiR = A.shape
        assert dA == d

        # choose operator on physical index
        X = op if k == i else np.eye(d, dtype=complex)

        # Update env:
        # env[a,b] * conj(A)[a,c,d] * X[c,e] * A[b,e,f] -> env_new[d,f]
        # env tracks both bra (a,d) and ket (b,f) bond indices
        env = np.einsum("ab,acd,ce,bef->df",
                        env,
                        np.conjugate(A),
                        X,
                        A,
                        optimize=True)
    # after last site, env is 1x1
    return float(env[0, 0].real)

def expect_one_site_via_state(mps, op, i):
    """Slow reference: reconstruct state and compute expectation (for testing)."""
    psi = mps_to_state(mps)
    L = len(mps)
    d = mps[0].shape[1]
    # build full operator (note: op is sparse, id2 is sparse)
    # Need to convert op to sparse if it isn't already
    from scipy.sparse import csr_matrix
    if not hasattr(op, 'toarray'):
        op = csr_matrix(op)
    ops = [id2]*L
    ops[i] = op
    O = kron_all(ops)
    return np.vdot(psi, O @ psi).real

### Test: expectation values vs full-state reference

In [18]:
# Check a few sites
for i in [0, L//2, L-1]:
    val_mps = expect_one_site(mps_full, sz, i)
    val_ref = expect_one_site_via_state(mps_full, sz, i)
    print(i, val_mps, val_ref, "diff=", abs(val_mps-val_ref))
    assert abs(val_mps - val_ref) < 1e-10
print("All expectation tests passed ✅")


0 -9.943434964299058e-15 -9.975961029473623e-15 diff= 3.252606517456513e-17
5 -1.2767564783189302e-14 -1.2830882190062454e-14 diff= 6.331740687315188e-17
9 -4.71844785465692e-15 -4.6004866582904924e-15 diff= 1.1796119636642762e-16
All expectation tests passed ✅


# 5. Compression study: truncation vs accuracy

Now you will truncate the MPS bond dimension to a chosen `chi_max` during the SVD conversion,
and quantify the induced errors.
The reference value should be computed using ED.
This is similar to th small plot you did at the end of point (2) of this notebook.

## Assignment F1
For several values of `chi_max` (e.g. 2, 4, 8, 16, ...):
1. build `mps_chi` from the ED ground state with truncation
2. compute:
   - fidelity with ED: $|\langle \psi_{ED} | \psi_{\chi} \rangle|^2$
   - energy error: $|E_{\chi} - E_{ED}|$
   - entanglement profile error
3. Repeat for different `h` values: deep paramagnet (h large), deep ferromagnet (h small), near critical.
   - when is small `chi` enough? Try to understand if there is a pattern
4. Repeat for different system sizes. Do larger systems require larger bond dimensions?


In [ ]:
def energy_via_state(psi, L, J=1.0, h=1.0):
    # todo...

def fidelity(psi_ref, psi):
    # phase-invariant overlap
    # todo...

# --- Your experiment ---
chis = [1, 2, 4, 8, 16, 32]
results = []

for chi in chis:
    mps_chi, sch_chi = state_to_mps(psi0_ed, L=L, d=2, chi_max=chi)
    psi_chi = mps_to_state(mps_chi)
    # phase-align for stability
    ph = np.vdot(psi_chi, psi0_ed)
    if abs(ph) > 0:
        psi_chi = psi_chi * (ph/abs(ph))

    F = fidelity(psi0_ed, psi_chi)
    E = energy_via_state(psi_chi, L=L, J=J, h=h)
    results.append((chi, F, abs(E - E0_ed)))

results

[(1, 0.21660304977862876, np.float64(7.727426723563319)),
 (2, 0.994096596018884, np.float64(0.04875699079055984)),
 (4, 0.999996535609478, np.float64(3.0281482192862086e-05)),
 (8, 0.9999999997424072, np.float64(2.697342793567259e-09)),
 (16, 1.0000000000000013, np.float64(3.375077994860476e-14)),
 (32, 1.0, np.float64(1.0658141036401503e-14))]

# 6. Computing expectation values with Matrix Product Operators (MPOs)

So far you computed single-site expectation values like $\langle \sigma_i^z \rangle$ using `expect_one_site`.
But what about **sums of operators**, like the **total energy** or **total magnetization**?

$$
\langle M_z \rangle = \langle \psi | \sum_{i=1}^{L} \sigma_i^z | \psi \rangle
$$

$$
\langle H \rangle = \langle \psi | H | \psi \rangle
$$

You *could* loop over sites for magnetization, but for the energy you have **two-site terms** that are harder to handle site-by-site.

A more general approach: represent the operator as a **Matrix Product Operator (MPO)**.

## Matrix Product Operators

An MPO is like an MPS, but with **two** physical indices per site:
- Each tensor `W[i]` has shape `(D_left, d_out, d_in, D_right)`
- Contracting an MPO with an MPS (from both sides) gives $\langle \psi | \hat{O} | \psi \rangle$

For open boundaries:
- `W[0]` has `D_left = 1`
- `W[L-1]` has `D_right = 1`

For a sum like $\hat{O} = \sum_i \hat{o}_i$ (e.g., magnetization), the MPO has a simple structure:
- It "passes through" the identity on most sites
- "Applies" $\hat{o}_i$ when reaching site $i$
- Accumulates all contributions

Building MPOs by hand is **tedious and error-prone**, especially for Hamiltonians with multiple terms.
We will **provide** the MPO builders for you. 
Your job is to **use them efficiently**.

In general, there exists libraries to handle MPO construction for you, which is why I won't be torturing everyone of you with this. However, if you feel brave, it's actually an interesting exercise to understand how to build the MPO represntation of an arbitrary hamiltonian by yourself.

**For the assignment/exercise, look after the definitions**

In [ ]:
# ======================================================
# PROVIDED: MPO builders for magnetization and TFIM energy
# ======================================================

def mpo_magnetization_z(L):
    """
    MPO for total magnetization M_z = sum_i sigma^z_i.
    
    Returns
    -------
    mpo : list[np.ndarray]
        W[i] has shape (D_left, d_out, d_in, D_right) with d_out=d_in=2
    """
    # For a sum of local operators, MPO bond dimension is 2:
    # [I, sigma^z] at each site
    # W structure:
    #   W[0] = [I, sigma^z]  (shape 1 x 2 x 2 x 2)
    #   W[i] = [[I,        0],
    #           [sigma^z,  I]]  (shape 2 x 2 x 2 x 2)
    #   W[L-1] = [[sigma^z],
    #              [I]]  (shape 2 x 2 x 2 x 1)
    
    sz_arr = sz.toarray() if hasattr(sz, 'toarray') else np.array(sz, dtype=complex)
    I = np.eye(2, dtype=complex)
    
    mpo = []
    
    # First tensor
    W0 = np.zeros((1, 2, 2, 2), dtype=complex)
    W0[0, :, :, 0] = I       # pass-through
    W0[0, :, :, 1] = sz_arr  # start accumulating
    mpo.append(W0)
    
    # Middle tensors
    for i in range(1, L-1):
        W = np.zeros((2, 2, 2, 2), dtype=complex)
        W[0, :, :, 0] = I       # pass-through
        W[0, :, :, 1] = sz_arr  # add sigma^z at this site
        W[1, :, :, 1] = I       # carry forward previous contributions
        mpo.append(W)
    
    # Last tensor
    WL = np.zeros((2, 2, 2, 1), dtype=complex)
    WL[0, :, :, 0] = sz_arr  # add sigma^z at last site
    WL[1, :, :, 0] = I       # accumulate all previous
    mpo.append(WL)
    
    return mpo


def mpo_tfim(L, J=1.0, h=1.0):
    """
    MPO for the TFIM Hamiltonian (open boundaries):
    H = -J * sum_{i=1}^{L-1} sigma^z_i sigma^z_{i+1} - h * sum_{i=1}^{L} sigma^x_i
    
    Returns
    -------
    mpo : list[np.ndarray]
        W[i] has shape (D_left, d_out, d_in, D_right) with d_out=d_in=2
    """
    # MPO bond dimension is 3 for nearest-neighbor Hamiltonian:
    # [I, sigma^z, -h*sigma^x - J*sigma^z] structure
    
    sz_arr = sz.toarray() if hasattr(sz, 'toarray') else np.array(sz, dtype=complex)
    sx_arr = sx.toarray() if hasattr(sx, 'toarray') else np.array(sx, dtype=complex)
    I = np.eye(2, dtype=complex)
    
    mpo = []
    
    # First tensor: start the MPO chain
    W0 = np.zeros((1, 2, 2, 3), dtype=complex)
    W0[0, :, :, 0] = -h * sx_arr          # local sx term
    W0[0, :, :, 1] = -J * sz_arr          # prepare for sz sz term
    W0[0, :, :, 2] = I                    # pass-through
    mpo.append(W0)
    
    # Middle tensors: transfer matrix structure
    for i in range(1, L-1):
        W = np.zeros((3, 2, 2, 3), dtype=complex)
        W[2, :, :, 0] = -h * sx_arr       # local sx term
        W[1, :, :, 0] = sz_arr            # complete the sz_i sz_{i+1} term
        W[2, :, :, 1] = -J * sz_arr       # prepare for next sz sz term
        W[2, :, :, 2] = I                 # pass-through
        W[0, :, :, 0] = I                 # collect finished terms
        mpo.append(W)
    
    # Last tensor: terminate the MPO chain
    WL = np.zeros((3, 2, 2, 1), dtype=complex)
    WL[2, :, :, 0] = -h * sx_arr          # local sx term at last site
    WL[1, :, :, 0] = sz_arr               # complete last sz sz term
    WL[0, :, :, 0] = I                    # collect all terms
    mpo.append(WL)
    
    return mpo


# Create MPOs for our current system
mpo_mz = mpo_magnetization_z(L)
mpo_H = mpo_tfim(L, J=J, h=h)

print(f"Magnetization MPO: L={L}, bond dimensions = {[W.shape for W in mpo_mz]}")
print(f"TFIM Hamiltonian MPO: L={L}, bond dimensions = {[W.shape for W in mpo_H]}")

## TODO: implement `expect_mpo(mps, mpo) -> scalar`

This function should compute $\langle \psi | \hat{O} | \psi \rangle$ where:
- `mps` is a list of tensors `A[i]` with shape `(chi_left, d, chi_right)`
- `mpo` is a list of tensors `W[i]` with shape `(D_left, d_out, d_in, D_right)`

### Algorithm: left-to-right contraction

You need to contract a 3-layer tensor network:
```
    bra:  conj(A[0])---conj(A[1])---conj(A[2])---...
               |            |            |
    MPO:     W[0]--------W[1]--------W[2]-------...
               |            |            |
    ket:      A[0]--------A[1]--------A[2]-------...
```

You should use the environment contraction algorithm we discussed in class. The sketch of the algorithm is roughly as follows:
1. Start with the left environment tensor `L` as a scalar (stored as a 3-tensor of shape `(1, 1, 1)` for the three bond indices: bra-bond, MPO-bond, ket-bond)
2. For each site `i`, update `L` by contracting:
   - previous `L[a, b, c]`
   - `conj(A[i])[a, s', d]`
   - `W[i][b, s', s, e]`
   - `A[i][c, s, f]`
   - Result: `L_new[d, e, f]`
3. After the last site, `L` should be shape `(1, 1, 1)` – extract the scalar.

**Hints:**
- Use `np.einsum` or `np.tensordot` for the contractions
- Be careful about index ordering!
- The physical indices `s` (ket) and `s'` (bra) are summed over
- Test against a known value (see test cells below)

In [ ]:
def expect_mpo(mps, mpo):
    """
    Compute <psi| O |psi> where O is represented as an MPO.
    
    Parameters
    ----------
    mps : list[np.ndarray]
        A[i] has shape (chi_left, d, chi_right)
    mpo : list[np.ndarray]
        W[i] has shape (D_left, d_out, d_in, D_right)
        
    Returns
    -------
    expectation : float
        Real part of the expectation value
    """
    # TODO: implement the left-to-right contraction
    raise NotImplementedError

### Test: Validate MPO expectation values

We'll test your `expect_mpo` function against known values:
1. **Magnetization**: Compare to summing individual site magnetizations
2. **Energy**: Compare to the ED ground state energy

In [ ]:
# Test 1: Total magnetization via MPO vs summing individual sites
mz_mpo = expect_mpo(mps_full, mpo_mz)
mz_sum = sum(expect_one_site(mps_full, sz, i) for i in range(L))

print(f"Magnetization via MPO: {mz_mpo:.12f}")
print(f"Magnetization via sum:  {mz_sum:.12f}")
print(f"Difference: {abs(mz_mpo - mz_sum):.2e}")
assert abs(mz_mpo - mz_sum) < 1e-10, "Magnetization mismatch!"
print("✅ Magnetization test passed\n")

# Test 2: Energy via MPO vs ED reference
energy_mpo = expect_mpo(mps_full, mpo_H)
energy_ed = E0_ed

print(f"Energy via MPO: {energy_mpo:.12f}")
print(f"Energy via ED:  {energy_ed:.12f}")
print(f"Difference: {abs(energy_mpo - energy_ed):.2e}")
assert abs(energy_mpo - energy_ed) < 1e-10, "Energy mismatch!"
print("✅ Energy test passed")

print("\n" + "="*50)
print("All MPO expectation value tests passed! ✅")
print("="*50)

In [ ]:
# =========================
# SOLUTION (G1): expect_mpo
# =========================
def expect_mpo(mps, mpo):
    """
    Compute <psi| O |psi> via left-to-right contraction.
    """
    L = len(mps)
    assert len(mpo) == L, "MPS and MPO must have same length"
    
    # Initialize left environment: scalar (all bonds are 1 at the start)
    env = np.ones((1, 1, 1), dtype=complex)
    
    for i in range(L):
        A = np.asarray(mps[i], dtype=complex)      # (chi_L, d, chi_R)
        W = np.asarray(mpo[i], dtype=complex)      # (D_L, d_out, d_in, D_R)
        
        # Contract the network:
        # env[a,b,c] * conj(A)[a,s',d] * W[b,s',s,e] * A[c,s,f] -> env_new[d,e,f]
        env = np.einsum('abc,asd,bste,csf->def',
                        env,
                        np.conj(A),
                        W,
                        A,
                        optimize=True)
    
    # After all sites, env should be (1, 1, 1)
    assert env.shape == (1, 1, 1), f"Final environment shape should be (1,1,1), got {env.shape}"
    
    return float(env[0, 0, 0].real)

# 7. Bonus section — Designing an MPS class to make your life easier

So far we've represented an MPS as a plain Python list of numpy arrays. This works, but as your code grows it becomes a bit error prone, as it's easy to forget the size or bond dimension of an MPS, and it's hard to remember how to operate on it.

The standard approach to solve those issues is to **wrap the MPS data in a class**, and defining a clean set of methods (aka, functions of the class) that operate on the data within the class

This helps you:
- **Group related data** that make up the MPS (tensors, bond dimensions, physical dimensions) into a single object, and always **move them together**. 
- **Define a clear interface**, because you can define methods like `MPS.expect(operator)` or `MPS.normalize()` or `MPS.entanglement_entropy()` which make the physics-part of the code much more readable and self-documenting. Moreover, if you pass it to someone else, it will be easier for them to understand , as they could just 'accept' that your .expect method works correctly and just look at other parts of it.
- **Validate input data:** In the constructor of the class, which is the `__init__(self, ...)` method, you can validate that all tensors have the correct shape (eg, matching bond dimension, match physical hilbert space size...) so that you catch errors earlier.
- **Easy to extend:** Want to add compression? Just add a `.compress(chi_max)` method instead of rewriting functions. And it will be easily discoverable just by pressing tab like `MPS.[\tab\tab\tab]`.
- **You can integrate with built in python functions:** By defining methods like `__add__` or `__matmul__` you can make your class work with `+` or `@` operators, which can be very nice. 

The tradeoff of encapsulating code in a class is that it takes some upfront design effort which is not always worth it. Usually, we prototype algorithms as 'free functions' and then when we know it works, we try to put it together in a class. 
Usually, if we just have a few classes in our code, there are no issues, but if we start to have plenty, their interaction can easily becomes complex.

## Bonus TODO: Implement a basic `MPS` class

Design a class with the following API:

### Constructor
implemented by defining the `__init__(self, arguments...):` method
```python
mps = MPS(tensors)
```
- Takes a list of tensors (each with shape `(chi_left, d, chi_right)`)
- Validates:
  - All tensors have 3 dimensions
  - Physical dimensions are consistent
  - Bond dimensions match between consecutive tensors
  - First tensor has `chi_left=1`, last has `chi_right=1`

### Properties (read-only)
implemented by defining a method and decorating it with `@property`.

- `mps.L`: number of sites
- `mps.d`: physical dimension (e.g., 2 for spin-1/2)
- `mps.bond_dims`: list of bond dimensions `[chi_0, chi_1, ..., chi_L]` where `chi_0 = chi_L = 1`
- `mps.max_bond_dim`: maximum bond dimension

### Methods
- `mps.to_state()`: return the full state vector (contracts the MPS)
- `mps.norm()`: return the norm `sqrt(<psi|psi>)`
- `mps.normalize()`: normalize the MPS in-place (divide by norm)
- `mps.expect_one_site(op, i)`: expectation value of single-site operator at site `i`
- `mps.expect_mpo(mpo)`: expectation value using an MPO

### Optional (advanced)
- `mps.copy()`: return a deep copy
- `mps.entanglement_entropy()`: return list of entanglement entropies at each cut
- `mps.compress(chi_max)`: truncate bond dimensions to `chi_max`

---

### Hints
- Use `@property` decorator for read-only properties
- Store tensors internally as `self._tensors` (list of arrays) so that `self.tensors` (the property) is read only
- Reuse the functions you already implemented (`mps_to_state`, `expect_one_site`, etc.)
- Make the constructor strict: better to fail early than produce garbage results

In [ ]:
class MPS:
    """
    Matrix Product State for open boundary conditions.
    
    Attributes
    ----------
    L : int
        Number of sites
    d : int
        Physical dimension
    bond_dims : list[int]
        Bond dimensions [chi_0, chi_1, ..., chi_L] with chi_0 = chi_L = 1
    max_bond_dim : int
        Maximum bond dimension
    """
    
    def __init__(self, tensors):
        """
        Initialize MPS from a list of tensors.
        
        Parameters
        ----------
        tensors : list[np.ndarray]
            Each tensor has shape (chi_left, d, chi_right)
        """
        # TODO: Validate and store tensors
        # TODO: Check shapes, bond dimensions, boundaries
        raise NotImplementedError
    
    @property
    def L(self):
        """Number of sites."""
        # TODO: implement
        raise NotImplementedError
    
    @property
    def d(self):
        """Physical dimension."""
        # TODO: implement
        raise NotImplementedError
    
    @property
    def bond_dims(self):
        """List of bond dimensions [chi_0, ..., chi_L]."""
        # TODO: implement
        raise NotImplementedError
    
    @property
    def max_bond_dim(self):
        """Maximum bond dimension."""
        # TODO: implement
        raise NotImplementedError
    
    def to_state(self):
        """Contract MPS to full state vector."""
        # TODO: implement (use mps_to_state)
        raise NotImplementedError
    
    def norm(self):
        """Compute norm of the MPS."""
        # TODO: implement
        raise NotImplementedError
    
    def normalize(self):
        """Normalize the MPS in-place."""
        # TODO: implement
        raise NotImplementedError
    
    def expect_one_site(self, op, i):
        """Expectation value of single-site operator at site i."""
        # TODO: implement (use expect_one_site function)
        raise NotImplementedError
    
    def expect_mpo(self, mpo):
        """Expectation value using an MPO."""
        # TODO: implement (use expect_mpo function)
        raise NotImplementedError
    
    def __repr__(self):
        """String representation."""
        return f"MPS(L={self.L}, d={self.d}, max_chi={self.max_bond_dim})"

### Test: Validate MPS class

Test your `MPS` class implementation with the following checks:
1. Constructor validation (should raise errors for invalid inputs)
2. Properties return correct values
3. Methods produce correct results compared to function-based implementation

In [ ]:
# Create MPS instance from our ED ground state
psi_mps = MPS(mps_full)

print("="*60)
print("Testing MPS class")
print("="*60)

# Test 1: Properties
print(f"\n1. Properties:")
print(f"   L = {psi_mps.L} (expected {L})")
assert psi_mps.L == L
print(f"   d = {psi_mps.d} (expected 2)")
assert psi_mps.d == 2
print(f"   bond_dims = {psi_mps.bond_dims}")
assert len(psi_mps.bond_dims) == L + 1
assert psi_mps.bond_dims[0] == 1
assert psi_mps.bond_dims[-1] == 1
print(f"   max_bond_dim = {psi_mps.max_bond_dim}")
print("   ✅ All properties correct")

# Test 2: to_state method
print(f"\n2. State reconstruction:")
psi_from_class = psi_mps.to_state()
psi_from_func = mps_to_state(mps_full)
err = np.linalg.norm(psi_from_class - psi_from_func)
print(f"   Difference from function: {err:.2e}")
assert err < 1e-12
print("   ✅ to_state() works correctly")

# Test 3: norm method
print(f"\n3. Norm:")
norm_val = psi_mps.norm()
expected_norm = np.linalg.norm(psi_from_class)
print(f"   norm() = {norm_val:.12f}")
print(f"   expected = {expected_norm:.12f}")
assert abs(norm_val - expected_norm) < 1e-12
print("   ✅ norm() works correctly")

# Test 4: expect_one_site method
print(f"\n4. Single-site expectation values:")
for i in [0, L//2, L-1]:
    val_class = psi_mps.expect_one_site(sz, i)
    val_func = expect_one_site(mps_full, sz, i)
    print(f"   Site {i}: class={val_class:.10f}, func={val_func:.10f}, diff={abs(val_class-val_func):.2e}")
    assert abs(val_class - val_func) < 1e-12
print("   ✅ expect_one_site() works correctly")

# Test 5: expect_mpo method
print(f"\n5. MPO expectation values:")
energy_class = psi_mps.expect_mpo(mpo_H)
energy_func = expect_mpo(mps_full, mpo_H)
print(f"   Energy (class): {energy_class:.12f}")
print(f"   Energy (func):  {energy_func:.12f}")
print(f"   Difference: {abs(energy_class - energy_func):.2e}")
assert abs(energy_class - energy_func) < 1e-12
print("   ✅ expect_mpo() works correctly")

# Test 6: Constructor validation (should fail for bad inputs)
print(f"\n6. Constructor validation:")
try:
    # Bad: mismatched bond dimensions
    bad_tensors = [np.ones((1, 2, 3)), np.ones((2, 2, 1))]  # bond 3 != bond 2
    MPS(bad_tensors)
    print("   ❌ Should have raised an error for mismatched bonds")
    assert False
except (ValueError, AssertionError) as e:
    print(f"   ✅ Correctly rejected mismatched bonds: {type(e).__name__}")

try:
    # Bad: first tensor doesn't have chi_left=1
    bad_tensors = [np.ones((2, 2, 2)), np.ones((2, 2, 1))]
    MPS(bad_tensors)
    print("   ❌ Should have raised an error for chi_left != 1")
    assert False
except (ValueError, AssertionError) as e:
    print(f"   ✅ Correctly rejected bad boundary: {type(e).__name__}")

print("\n" + "="*60)
print("All MPS class tests passed! ✅")
print("="*60)

In [ ]:
# =========================
# SOLUTION (H1): MPS class
# =========================

class MPS:
    """
    Matrix Product State for open boundary conditions.
    
    Attributes
    ----------
    L : int
        Number of sites
    d : int
        Physical dimension
    bond_dims : list[int]
        Bond dimensions [chi_0, chi_1, ..., chi_L] with chi_0 = chi_L = 1
    max_bond_dim : int
        Maximum bond dimension
    """
    
    def __init__(self, tensors):
        """
        Initialize MPS from a list of tensors.
        
        Parameters
        ----------
        tensors : list[np.ndarray]
            Each tensor has shape (chi_left, d, chi_right)
        """
        if not isinstance(tensors, list) or len(tensors) == 0:
            raise ValueError("tensors must be a non-empty list")
        
        # Convert to numpy arrays and validate shapes
        self._tensors = [np.asarray(t, dtype=complex) for t in tensors]
        
        # Check all tensors are 3D
        for i, t in enumerate(self._tensors):
            if t.ndim != 3:
                raise ValueError(f"Tensor {i} has {t.ndim} dimensions, expected 3")
        
        # Check physical dimensions are consistent
        d_values = [t.shape[1] for t in self._tensors]
        if not all(d == d_values[0] for d in d_values):
            raise ValueError(f"Inconsistent physical dimensions: {d_values}")
        self._d = d_values[0]
        
        # Check bond dimensions match
        self._L = len(self._tensors)
        for i in range(self._L - 1):
            chi_R = self._tensors[i].shape[2]
            chi_L_next = self._tensors[i+1].shape[0]
            if chi_R != chi_L_next:
                raise ValueError(
                    f"Bond mismatch between sites {i} and {i+1}: "
                    f"{chi_R} != {chi_L_next}"
                )
        
        # Check boundary conditions
        if self._tensors[0].shape[0] != 1:
            raise ValueError(f"First tensor must have chi_left=1, got {self._tensors[0].shape[0]}")
        if self._tensors[-1].shape[2] != 1:
            raise ValueError(f"Last tensor must have chi_right=1, got {self._tensors[-1].shape[2]}")
    
    @property
    def L(self):
        """Number of sites."""
        return self._L
    
    @property
    def d(self):
        """Physical dimension."""
        return self._d
    
    @property
    def bond_dims(self):
        """List of bond dimensions [chi_0, ..., chi_L]."""
        dims = [self._tensors[0].shape[0]]  # chi_0 = 1
        for t in self._tensors:
            dims.append(t.shape[2])  # chi_right of each tensor
        return dims
    
    @property
    def max_bond_dim(self):
        """Maximum bond dimension."""
        return max(self.bond_dims)
    
    def to_state(self):
        """Contract MPS to full state vector."""
        return mps_to_state(self._tensors)
    
    def norm(self):
        """Compute norm of the MPS."""
        # Efficient: contract <psi|psi> without building full state
        # For now, use the state method (fine for small systems)
        psi = self.to_state()
        return float(np.linalg.norm(psi))
    
    def normalize(self):
        """Normalize the MPS in-place."""
        nrm = self.norm()
        if nrm == 0:
            raise ValueError("Cannot normalize zero state")
        # Divide first tensor by norm (arbitrary choice)
        self._tensors[0] = self._tensors[0] / nrm
    
    def expect_one_site(self, op, i):
        """Expectation value of single-site operator at site i."""
        return expect_one_site(self._tensors, op, i)
    
    def expect_mpo(self, mpo):
        """Expectation value using an MPO."""
        return expect_mpo(self._tensors, mpo)
    
    def copy(self):
        """Return a deep copy of the MPS."""
        import copy
        return MPS([copy.deepcopy(t) for t in self._tensors])
    
    def __repr__(self):
        """String representation."""
        return f"MPS(L={self.L}, d={self.d}, max_chi={self.max_bond_dim})"